In [1]:
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models

from tqdm import tqdm

In [2]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [3]:
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset


train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_dataset = HymenopteraDataset(
                    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = HymenopteraDataset(
                    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')

batch_size = 32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


# 네트워크 모델 작성

In [4]:
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

net.train()

print('네트워크 설정 완료: 학습된 가중치를 로드하고 훈련 모드로 설정했습니다.')

/home/skang/Documents/Pytorch_tutorials/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/skang/Documents/Pytorch_tutorials/venv/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


네트워크 설정 완료: 학습된 가중치를 로드하고 훈련 모드로 설정했습니다.


# 손실 함수 정의

In [5]:
criterion = nn.CrossEntropyLoss()

# 최적화 방법 설정

In [6]:
net.named_parameters()

<generator object Module.named_parameters at 0x7f75e1349660>

In [7]:
# 파인 튜닝으로 학습할 파라미터를 params_to_update 변수의 1~3에 저장한다.

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 학습시킬 층의 파라미터명을 지정
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1에 저장: ", name)
    
    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2에 저장: ", name)
    
    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3에 저장")
    else:
        param.requires_grad = False
        print("경사 계산없음. 학습하지 않음: ", name)

params_to_update_1에 저장:  features.0.weight
params_to_update_1에 저장:  features.0.bias
params_to_update_1에 저장:  features.2.weight
params_to_update_1에 저장:  features.2.bias
params_to_update_1에 저장:  features.5.weight
params_to_update_1에 저장:  features.5.bias
params_to_update_1에 저장:  features.7.weight
params_to_update_1에 저장:  features.7.bias
params_to_update_1에 저장:  features.10.weight
params_to_update_1에 저장:  features.10.bias
params_to_update_1에 저장:  features.12.weight
params_to_update_1에 저장:  features.12.bias
params_to_update_1에 저장:  features.14.weight
params_to_update_1에 저장:  features.14.bias
params_to_update_1에 저장:  features.17.weight
params_to_update_1에 저장:  features.17.bias
params_to_update_1에 저장:  features.19.weight
params_to_update_1에 저장:  features.19.bias
params_to_update_1에 저장:  features.21.weight
params_to_update_1에 저장:  features.21.bias
params_to_update_1에 저장:  features.24.weight
params_to_update_1에 저장:  features.24.bias
params_to_update_1에 저장:  features.26.weight
params_to_update_1

In [8]:
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3},
    
], momentum=0.9)

# 학습 및 검증 실시

In [9]:
# 모델을 학습시키는 함수를 작성
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    
    
    # 초기 설정
    # GPU가 사용 가능한지 확인
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("사용 장치: ", device)
    
    # 네트워크를 GPU로
    net.to(device)
    
    torch.backends.cudnn.benchmark = True
    
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print("-------------")
        
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() # 모델을 훈련모드로
            else:
                net.eval() # 모델을 검증모드로
                
            epoch_loss = 0.0  # epochs 손실의 합
            epoch_corrects = 0 # epoch 정답 수
            
            if (epoch == 0) and (phase == 'train'): # 학습 전 처음에는 validation 모드로 실행
                continue
                
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                
                # GPU가 사용 가능하면 GPU에 데이터 보내기
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                # optimizer를 초기화
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels) # 손실 계산
                    _, pred = torch.max(outputs, 1)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(pred == labels.data)
                    
                epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
                epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
                
                print(f'{phase} Loss : {epoch_loss:.4f} Acc : {epoch_acc:.4f}')
    

In [10]:
num_epochs = 2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 장치:  cuda:0
Epoch 1/2
-------------


 20%|██        | 1/5 [00:02<00:11,  2.87s/it]

val Loss : 0.1636 Acc : 0.0980


 40%|████      | 2/5 [00:03<00:04,  1.40s/it]

val Loss : 0.1487 Acc : 0.2092


...

val Loss : 0.0471 Acc : 0.6013


 80%|████████  | 4/5 [00:01<00:00,  2.54it/s]

val Loss : 0.0457 Acc : 0.7974


100%|██████████| 5/5 [00:01<00:00,  2.62it/s]

val Loss : 0.0229 Acc : 0.9608


# 학습한 네트워크의 정장 및 로드

In [11]:
# Pytorch 네트워크 파라미터 저장
save_path = './weight_fine_tuning.pth'
torch.save(net.state_dict(), save_path)

In [12]:
# Pytorch 네트워크 파라미터 로드
load_path = './weight_fine_tuning.pth'
load_weights = torch.load(load_path)
net.load_state_dict(load_weights) # 클래스 인스턴스를 만든 후 load_state_dict 함수를 이용하여 인자 전달.

# GPU 상에 저장된 가중치를 CPU에 로드할 경우
load_weights = torch.load(load_path, map_location={'cuda:0': 'cpu'})
net.load_state_dict(load_weights) 

<All keys matched successfully>